In [1]:
import html

import pandas as pd
import numpy as np

from sklearn.neighbors import NearestNeighbors
import os, sys
import re

from sklearn.model_selection import train_test_split

In [2]:
#cwdir = os.path.abspath('')
book_path = r'./dataset/BX-Books.csv'
user_path = r'./dataset/BX-Users.csv'
rating_path = r'./dataset/BX-Book-Ratings.csv'

In [3]:
import os
os.getcwd()

'C:\\Users\\Legion\\Downloads\\b\\1INTRO2AI\\GithubDesktop\\book-rec'

In [4]:
# Fix the &amp; and other HTML escape sequences
raw_book_path = book_path
book_path = r'./BX-Books-HTMLfixed.csv'
with open(raw_book_path, 'r', encoding='latin-1') as f, open(book_path, 'w') as g:
    content = html.unescape(f.read())
    g.write(content)

In [5]:
books = pd.read_csv(book_path, encoding='latin-1' , on_bad_lines='skip' , sep=';' , low_memory=False, escapechar='\\')
users = pd.read_csv(user_path, encoding='latin-1' , on_bad_lines='skip' , sep=';' , low_memory=False, escapechar='\\')
ratings = pd.read_csv(rating_path, encoding='latin-1' , on_bad_lines='skip' , sep=';' , low_memory=False, escapechar='\\')

In [6]:
pd.set_option('display.max_colwidth', None)

In [7]:
users.dtypes

User-ID       int64
Location     object
Age         float64
dtype: object

In [8]:
ratings.dtypes

User-ID         int64
ISBN           object
Book-Rating     int64
dtype: object

In [9]:
# Data Preprocessing
books.loc[books['Book-Author'].isnull(), 'Book-Author'] = "No author"
books.loc[books['Publisher'].isnull(), 'Book-Author'] = "Other"

In [10]:
print(users.shape)
users.head()
print(sorted(users['Age'].unique()))

(278858, 3)
[nan, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 113.0, 114.0, 115.0, 116.0, 118.0, 119.0, 123.0, 124.0, 127.0, 128.0, 132.0, 133.0, 136.0, 137.0, 138.0, 140.0, 141.0, 143.0, 146.0, 147.0, 148.0, 151.0, 152.0, 156.0, 157.0, 159.0, 162.0, 168.0, 172.0, 175.0, 183.0, 186.0, 189.0, 199.0, 200.0, 201.0, 204.0, 207.0, 208.0, 209.0, 210.0, 212.0, 219.0, 220.0, 

In [11]:
users.loc[(users['Age'] < 5) | (users['Age'] > 100), 'Age'] = pd.NA
users['Age'] = users['Age'].fillna(users['Age'].mean())
users['Age'] = users['Age'].astype(np.int64)

In [12]:
ratings['Book-Rating'].unique()

array([ 0,  5,  3,  6,  8,  7, 10,  9,  4,  1,  2], dtype=int64)

In [13]:
# print(users.shape[0] * books.shape[0])
# Check if people rated is in database
# r_existpeople = ratings[ratings['User-ID'].isin(users['User-ID'])]
# print(r_existpeople.shape)

# Check if books rated is in database
r_existbook = ratings[ratings['ISBN'].isin(books['ISBN'])]
r_existbook

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149774,276704,0876044011,0
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10


In [14]:
ratings_nonzero = r_existbook[r_existbook['Book-Rating'] != 0]
# ratings_explicit
ratings_allzero = r_existbook[r_existbook['Book-Rating'] == 0]
# ratings_implicit
ratings_nonzero

,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
8,276744,038550120X,7
16,276747,0060517794,9
...,...,...,...
1149771,276704,0743211383,7
1149773,276704,0806917695,5
1149775,276704,1563526298,9
1149777,276709,0515107662,10


In [16]:
# reducing to user rating at least 10 books and books have 20 ratings
# Only considering explicit rating (rating != 0)
used_rating = ratings_nonzero

count_books = used_rating['ISBN'].value_counts()
count_users = used_rating['User-ID'].value_counts()
print(count_books.shape)
print(count_users.shape)

used_rating = used_rating[used_rating['User-ID'].isin(count_users[count_users > 100].index)]
used_rating = used_rating[used_rating['ISBN'].isin(count_books[count_books > 50].index)]

print(used_rating)

(149842,)
(68092,)
         User-ID        ISBN  Book-Rating
1456      277427  002542730X           10
1474      277427  0061009059            9
1522      277427  0316776963            8
1543      277427  0345413903           10
1581      277427  0385486804            9
...          ...         ...          ...
1146735   275970  038572179X            9
1146737   275970  0385722206           10
1146809   275970  043936213X            9
1146820   275970  0440180295            9
1146852   275970  044651652X            6

[7386 rows x 3 columns]


In [17]:
X_train = used_rating

ratings_matrix = X_train.pivot(index = 'User-ID', columns = 'ISBN', values = 'Book-Rating')
print(ratings_matrix.shape)
ratings_matrix = ratings_matrix.fillna(0)

ratings_matrix

(419, 507)


ISBN,002542730X,0060096195,006016848X,0060173289,0060175400,0060199652,0060391626,0060392452,0060502258,0060915544,...,1558744630,1558745157,1559029838,1573225517,1573225789,1573227331,1573229326,1573229571,1592400876,1878424319
User-ID,,,,,,,,,,,,,,,,,,,,,
2033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0
4385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274061,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
274301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
275970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# KNN Function
#setting global variables
global metric,k
k=10
global_metric='cosine'
global_algorithm = 'brute'

In [19]:
def findksimilarusers(user_id, rate_matrix, metric=global_metric, algo=global_algorithm, k=k):
    similarities=[]
    indices=[]
    model_knn = NearestNeighbors(metric = metric, algorithm = algo)
    model_knn.fit(rate_matrix.values)
    
    loc = rate_matrix.index.get_loc(user_id)
    # Return k nearest neighbors' indices and distances to the user_id
    distances, indices = model_knn.kneighbors(rate_matrix.iloc[loc, :].values.reshape(1, -1), n_neighbors = k+1)
    
    # Get similarity level by (1 - differences)
    similarities = 1 - distances.flatten()
    indices = indices.flatten()
    
    return similarities, indices

In [20]:
#This function predicts rating for specified user-item combination based on user-based approach
def predict_userbased(user_id, item_id, rate_matrix, similarities, indices):
    prediction = 0
    similarity_sum = np.sum(similarities) - 1 # Minus the current user (similarity = 1)
    weighted_sum = 0
    product = 1

    item_loc = rate_matrix.columns.get_loc(item_id)
    user_loc = rate_matrix.index.get_loc(user_id)
    mean_rating = rate_matrix.iloc[user_loc, :].mean()

    # For each similar user:
    for i in range(0, len(indices)):
        if indices[i] == user_loc:
            continue;
        else:
            # Get the rating differences from the mean (of other items), multiply by similarities with the selected user
            ratings_diff = rate_matrix.iloc[indices[i], item_loc] - np.mean(rate_matrix.iloc[indices[i],:])
            product = ratings_diff * (similarities[i])
            weighted_sum = weighted_sum + product
    
    # weighted_sum = summation< (x - mean) * sim > / sumsim
                                                     
    #in case of very sparse datasets, using correlation metric for collaborative based approach may give negative ratings
    #which are handled here as below
    if prediction <= 0:
        prediction = 1   
    elif prediction >10:
        prediction = 10
    
    # Calculate rating by weighted mean of ratings and similarity score of k similar users
    prediction = int(round(mean_rating + (weighted_sum/similarity_sum)))
    # print ('\nPredicted rating for user {0} -> item {1}: {2}'.format(user_id,item_id,prediction))

    return prediction

In [21]:
def recommendedItem(user_id, rate_matrix, metric=global_metric, algorithm=global_algorithm, k=k):
    if (user_id not in rate_matrix.index.values) or type(user_id) is not int:
        print("User id should be a valid integer from this list : \n\n {}".format(re.sub('[\[\]]', '', np.array_str(rate_matrix.index.values))))
    else:
        prediction = []
        # Similar users based on cosine similarity
        similarities, indices = findksimilarusers(user_id, rate_matrix, metric, algorithm, k)
        
        total_book = rate_matrix.shape[1]
        for i in range(total_book):
            item_id = str(rate_matrix.columns[i])
            if (rate_matrix[item_id][user_id] != 0): #not rated already
                print("\r{0:<0.2f}% Finished".format(i*100/total_book), end='')
                prediction.append(predict_userbased(user_id, item_id, rate_matrix, similarities, indices))
            else:                    
                prediction.append(-1) #for books that user already rated
#         else:
#             print("Item-based not implemented yet")
        print("\r100.00% Finished")
        
        prediction = pd.Series(prediction)
        prediction = prediction.sort_values(ascending = False)
        print("...")
        
        # rmse_score = rmse_on_test_set(rate_matrix, similarities, indices)
        # print("RMSE Score: ", rmse_score)
        
        recommended = prediction[:10]
        print("\nFor the User-based ({0} - {1}) approach, the following books are recommended: \n".format(global_algorithm, global_metric))
        for i in range(len(recommended)):
            print("{0}. Rated {1}: {2}".format(i+1, prediction.values[i], books['Book-Title'][recommended.index[i]]))
                

In [23]:
print(len(ratings_matrix))
global_algorithm = 'brute'
global_metric = 'cosine'
recommendedItem(23902, ratings_matrix)

419
100.00% Finished
...

For the User-based (brute - cosine) approach, the following books are recommended: 

1. Rated 4: Angels & Demons
2. Rated 3: Purity in Death
3. Rated 3: Haifischfrauen.
4. Rated 3: The Deal
5. Rated 3: If I'd Known Then What I Know Now: Why Not Learn from the Mistakes of Others? : You Can't Afford to Make Them All Yourself
6. Rated 3: Born Confused
7. Rated 2: Seabiscuit: An American Legend
8. Rated 2: Bringing Down the House: The Inside Story of Six M.I.T. Students Who Took Vegas for Millions
9. Rated 2: My Antonia
10. Rated 2: L'home del sac


In [26]:
global_algorithm = 'brute'
global_metric = 'euclidean'
recommendedItem(23902, ratings_matrix, global_metric, global_algorithm)

100.00% Finished
...

For the User-based (brute - euclidean) approach, the following books are recommended: 

1. Rated 1: Seabiscuit: An American Legend
2. Rated 1: My Antonia
3. Rated 1: Powder and Patch
4. Rated 0: Every Breath You Take : A True Story of Obsession, Revenge, and Murder
5. Rated 0: Wicked: The Life and Times of the Wicked Witch of the West
6. Rated 0: If I'd Known Then What I Know Now: Why Not Learn from the Mistakes of Others? : You Can't Afford to Make Them All Yourself
7. Rated 0: The Therapeutic Touch: How to Use Your Hands to Help or to Heal
8. Rated 0: Affinity
9. Rated 0: Pride and Prejudice
10. Rated 0: The Deal


In [27]:
global_algorithm = 'brute'
global_metric = 'correlation'
recommendedItem(23902, ratings_matrix, global_metric, global_algorithm)

100.00% Finished
...

For the User-based (brute - correlation) approach, the following books are recommended: 

1. Rated 4: Angels & Demons
2. Rated 4: Haifischfrauen.
3. Rated 3: Purity in Death
4. Rated 3: The Deal
5. Rated 3: If I'd Known Then What I Know Now: Why Not Learn from the Mistakes of Others? : You Can't Afford to Make Them All Yourself
6. Rated 2: Essa efa (ColÂ¨lecciÃ³ El Mirall i el temps)
7. Rated 2: My Antonia
8. Rated 2: An Atmosphere of Eternity: Stories of India
9. Rated 2: Bringing Down the House: The Inside Story of Six M.I.T. Students Who Took Vegas for Millions
10. Rated 2: Contact


In [28]:
# Not counting zero rating
# Same 100 filter
used_rating = ratings_nonzero
rating_filter = ratings_nonzero

counts1 = rating_filter['User-ID'].value_counts()
counts = rating_filter['ISBN'].value_counts()
print(counts1)
print(counts)

used_rating = used_rating[used_rating['User-ID'].isin(counts1[counts1 >= 100].index)]
used_rating = used_rating[used_rating['ISBN'].isin(counts[counts >= 50].index)]

used_rating = used_rating[used_rating['Book-Rating'] != 0]

11676     6943
98391     5691
189835    1899
153662    1845
23902     1180
          ... 
114767       1
114771       1
114772       1
114777       1
276721       1
Name: User-ID, Length: 68092, dtype: int64
0316666343    707
0971880107    581
0385504209    487
0312195516    383
0060928336    320
             ... 
8420431249      1
840804978X      1
8408020854      1
355175537X      1
0806917695      1
Name: ISBN, Length: 149842, dtype: int64


In [29]:
used_rating = used_rating.sample(frac = 1)

ratings_matrix = used_rating.pivot(index = 'User-ID', columns = 'ISBN', values = 'Book-Rating')
userID = ratings_matrix.index
ISBN = ratings_matrix.columns
print(ratings_matrix.shape)
ratings_matrix = ratings_matrix.fillna(0)

print(used_rating)
ratings_matrix

(427, 531)
         User-ID        ISBN  Book-Rating
262639     60337  0142001740            8
224030     52350  0345339711            9
61278      12538  0380012863            7
174668     37950  0399501487            9
1104140   265115  0312983271            8
...          ...         ...          ...
507689    123608  0446672211            8
680568    165758  0553272535            5
1002403   240567  0440241537            5
284255     67840  0449005615           10
577325    138844  0671003755            7

[7660 rows x 3 columns]


ISBN,002542730X,0060096195,006016848X,0060173289,0060175400,0060199652,0060391626,0060392452,0060502258,0060915544,...,1558745157,1559029838,1573225517,1573225789,1573227331,1573229326,1573229571,1592400876,1844262553,1878424319
User-ID,,,,,,,,,,,,,,,,,,,,,
2033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
4385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274061,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
274301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
275970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
print(len(ratings_matrix))
global_algorithm = 'brute'
global_metric = 'cosine'
recommendedItem(23902, ratings_matrix)
counts1.loc[[23902]]

427
100.00% Finished
...
119    4
288    3
22     3
120    3
197    3
      ..
180   -1
179   -1
178   -1
177   -1
530   -1
Length: 531, dtype: int64

For the User-based (brute - cosine) approach, the following books are recommended: 

1. Rated 4: The Deal
2. Rated 3: Poisonwood Bible Edition Uk
3. Rated 3: If I'd Known Then What I Know Now: Why Not Learn from the Mistakes of Others? : You Can't Afford to Make Them All Yourself
4. Rated 3: All He Ever Wanted: A Novel
5. Rated 3: Hebt den Dachbalken hoch, Zimmerleute / Seymour wird vorgestellt.
6. Rated 3: Purity in Death
7. Rated 2: Martian Chronicles
8. Rated 2: EYE ON CRIME: HARDY BOYS #153
9. Rated 2: Peter Pan: The Original Story (Peter Pan)
10. Rated 2: If I Ever Get Back to Georgia, I'm Gonna Nail My Feet to the Ground


23902    1180
Name: User-ID, dtype: int64

In [31]:
counts1

11676     6943
98391     5691
189835    1899
153662    1845
23902     1180
          ... 
114767       1
114771       1
114772       1
114777       1
276721       1
Name: User-ID, Length: 68092, dtype: int64